<a href="https://colab.research.google.com/github/fininsight/nlp-deeplearning-tutorial/blob/master/05_Prac_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5장. CNN

#5-1. CNN

https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/images/cnn.ipynb?hl=ko#scrollTo=jKgyC5K_4O0d

## MNIST 데이터셋 다운로드하고 준비하기

In [1]:
#!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널
test_images = test_images.reshape((10000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## CNN 만들기

In [2]:
model = models.Sequential()
# 특징 추출 (Feature Extraction)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 32 * 3 * 3 + 32 = 320
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 32 * 64 * 3 * 3 + 64 = 18496
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 64 * 64 * 3 * 3 + 64 = 36928

# 분류 (Classification)
model.add(layers.Flatten()) # 576개 벡터로 Flatten
model.add(layers.Dense(64, activation='relu')) # 576 * 64 + 64 = 36928
model.add(layers.Dense(10, activation='softmax')) # 64 * 10 + 10 = 650

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

## 모델 컴파일과 훈련하기

In [3]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1400 - accuracy: 0.9571
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0446 - accuracy: 0.9865
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0319 - accuracy: 0.9904
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0246 - accuracy: 0.9925
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0191 - accuracy: 0.9934


## 모델 평가

In [4]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 1s - loss: 0.0313 - accuracy: 0.9907
0.9907000064849854


---

# 5-2. CNN for Sentence Classification

In [2]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

--2020-08-02 14:46:11--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-08-02 14:46:11--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  34.7MB/s    in 0.5s    

2020-08-02 14:46:12 (34.7 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("./ratings.txt",sep='\t').dropna()

In [4]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [5]:
import json
from tensorflow.keras import preprocessing

vocab_file = "vocab.json"
padding_size = 128
x_text, y = df["document"].tolist(), df["label"].tolist()

# Texts to sequences
text_preprocesser = preprocessing.text.Tokenizer(oov_token="<UNK>")
text_preprocesser.fit_on_texts(x_text)
x = text_preprocesser.texts_to_sequences(x_text)
word_dict = text_preprocesser.word_index
json.dump(word_dict, open(vocab_file, 'w'), ensure_ascii=False)
vocab_size = len(word_dict)
# max_doc_length = max([len(each_text) for each_text in x])
x = preprocessing.sequence.pad_sequences(x, maxlen=padding_size,
                                        padding='post', truncating='post')
print("Vocabulary size: {:d}".format(vocab_size))
print("Shape of train data: {}".format(np.shape(x)))

x_train, y_train = x, y

Vocabulary size: 369031
Shape of train data: (199992, 128)


In [6]:
    # parser.add_argument('-t', '--test_sample_percentage', default=0.1, type=float, help='The fraction of test data.(default=0.1)')
    # parser.add_argument('-p', '--padding_size', default=128, type=int, help='Padding size of sentences.(default=128)')
    # parser.add_argument('-e', '--embed_size', default=512, type=int, help='Word embedding size.(default=512)')
    # parser.add_argument('-f', '--filter_sizes', default='3,4,5', help='Convolution kernel sizes.(default=3,4,5)')
    # parser.add_argument('-n', '--num_filters', default=128, type=int, help='Number of each convolution kernel.(default=128)')
    # parser.add_argument('-d', '--dropout_rate', default=0.5, type=float, help='Dropout rate in softmax layer.(default=0.5)')
    # parser.add_argument('-c', '--num_classes', default=18, type=int, help='Number of target classes.(default=18)')
    # parser.add_argument('-l', '--regularizers_lambda', default=0.01, type=float, help='L2 regulation parameter.(default=0.01)')
    # parser.add_argument('-b', '--batch_size', default=64, type=int, help='Mini-Batch size.(default=64)')
    # parser.add_argument('--epochs', default=10, type=int, help='Number of epochs.(default=10)')
    # parser.add_argument('--fraction_validation', default=0.05, type=float, help='The fraction of validation.(default=0.05)')
    # parser.add_argument('--results_dir', default='./results/', type=str, help='The results dir including log, model, vocabulary and some images.(default=./results/)')


embed_size = 512 #단어 임베딩 사이즈
num_classes = 2 #클래스 갯수
num_filters = 128 #필터 개수 (필터사이즈와 같음)
filter_sizes = [3,4,5]
regularizers_lambda = 0.01 #L2 regulation parameter
dropout_rate =  0.5
feature_size = 128 #문장 시퀀스 길이
#vocab_size = 10000 #vocab 사이즈



In [7]:
from tensorflow import keras

inputs = keras.Input(shape=(feature_size,), name='input_data')
embed_initer = keras.initializers.RandomUniform(minval=-1, maxval=1)
embed = keras.layers.Embedding(vocab_size, embed_size,
                                embeddings_initializer=embed_initer,
                                input_length=feature_size,
                                name='embedding')(inputs)
                                
# single channel. If using real embedding, you can set one static
embed = keras.layers.Reshape((feature_size, embed_size, 1), name='add_channel')(embed)

pool_outputs = []
for filter_size in filter_sizes :
    filter_shape = (filter_size, embed_size)
    conv = keras.layers.Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                                data_format='channels_last', activation='relu',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                name='convolution_{:d}'.format(filter_size))(embed)
    max_pool_shape = (feature_size - filter_size + 1, 1)
    pool = keras.layers.MaxPool2D(pool_size=max_pool_shape,
                                  strides=(1, 1), padding='valid',
                                  data_format='channels_last',
                                  name='max_pooling_{:d}'.format(filter_size))(conv)
    pool_outputs.append(pool)

pool_outputs = keras.layers.concatenate(pool_outputs, axis=-1, name='concatenate')
pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(pool_outputs)
pool_outputs = keras.layers.Dropout(dropout_rate, name='dropout')(pool_outputs)

outputs = keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer='glorot_normal',
                              bias_initializer=keras.initializers.constant(0.1),
                              kernel_regularizer=keras.regularizers.l2(regularizers_lambda),
                              bias_regularizer=keras.regularizers.l2(regularizers_lambda),
                              name='dense')(pool_outputs)
model = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 512)     188943872   input_data[0][0]                 
__________________________________________________________________________________________________
add_channel (Reshape)           (None, 128, 512, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
convolution_3 (Conv2D)          (None, 126, 1, 128)  196736      add_channel[0][0]                
______________________________________________________________________________________________

In [9]:
import os
import time
import tensorflow as tf

batch_size = 64
epochs = 10
fraction_validation = 0.05

results_dir = "./result/"
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
timestamp = time.strftime("%Y-%m-%d-%H-%M", time.localtime(time.time()))
if not os.path.exists(os.path.join(results_dir, timestamp)) :
  os.mkdir(os.path.join(results_dir, timestamp))
  os.mkdir(os.path.join(results_dir, timestamp, 'log/'))


parallel_model = model#keras.utils.multi_gpu_model(model, gpus=1)
model.compile(tf.optimizers.Adam(), loss='categorical_crossentropy',
                        metrics=['accuracy'])
keras.utils.plot_model(model, show_shapes=True, to_file=os.path.join(results_dir, timestamp, "model.jpg"))
y_train = tf.one_hot(y_train, num_classes)
tb_callback = keras.callbacks.TensorBoard(os.path.join(results_dir, timestamp, 'log/'),
                                          histogram_freq=0.1, write_graph=True,
                                          write_grads=True, write_images=True,
                                          embeddings_freq=0.5, update_freq='batch')
history = parallel_model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
                              callbacks=[tb_callback], validation_split=fraction_validation, shuffle=True)
print("\nSaving model...")
keras.models.save_model(model, save_path)
pprint(history.history)

Epoch 1/10
 205/2969 [=>............................] - ETA: 47:47 - loss: 0.7230 - accuracy: 0.6104

KeyboardInterrupt: ignored

In [6]:
import scipy
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras


# TPU 초기화

print('Pandas: %s'%(pd.__version__))
print('Numpy: %s'%(np.__version__))
print('Scipy: %s'%(scipy.__version__))
print('Tensorflow: %s'%(tf.__version__))
print('Keras: %s'%(keras.__version__))

# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Pandas: 1.0.5
Numpy: 1.18.5
Scipy: 1.4.1
Tensorflow: 2.2.0
Keras: 2.3.0-tf
INFO:tensorflow:Initializing the TPU system: grpc://10.31.221.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.31.221.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.31.221.50:8470']
Number of accelerators:  8
